# Проект "Музыка больших городов"

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

На данных музыкального сервиса мы сравним поведение пользователей двух столиц.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**

Данные о поведении пользователей мы получим из файла `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Мы проверим данные на ошибки и оценим их влияние на исследование. Затем, на этапе предобработки поищем возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## 1. Обзор данных

In [ ]:
import pandas as pd #импортируем библиотеку pandas

In [ ]:
df = pd.read_csv('data.csv') #считываем данные из файла

In [ ]:
df.head(10) #ввыодим на экран первые 10 строк таблицы

In [ ]:
df.info() #изучаем общую информацию по таблице

Итак, в таблице семь столбцов. Тип данных во всех столбцах — `object`.

**Согласно документации к данным:**
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**В названиях колонок видны три нарушения стиля:**
1. Строчные буквы сочетаются с прописными.
2. Встречаются пробелы.
3. Слова не разделяются нижним подчёркиванием.

### Промежуточный вывод

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## 2. Предобработка данных

### 2.1 Стиль заголовков

In [2]:
df.columns #выводим на экран названия столбцов

NameError: name 'data' is not defined

In [1]:
df = df.rename(columns = {'  userID' : 'user_id', 'Track' : 'track', '  City  ' : 'city', 'Day' : 'day'}) 
#приводим названия столбцов к стилистически верным

NameError: name 'df' is not defined

In [ ]:
df.columns #проверяем результат

### 2.2 Пропуски значений

In [ ]:
pd.isna(df).sum() #подсчитываем сколько в таблице пропущенных значений

Не все пропущенные значения влияют на исследование. Так, в `track` (1231 пропуск) и `artist` (7203 пропуска) пропуски не важны для нашей работы. Достаточно заменить их явными обозначениями.

Но пропуски в `genre` (1198 пропусков) могут помешать сравнить музыкальные вкусы в Москве и Санкт-Петербурге. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Поэтому придётся:
* заполнить и эти пропуски явными обозначениями;
* оценить, насколько они повредят расчётам.

In [ ]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown') 
    #заменяем пропущенные значения явным обозначением

In [ ]:
df.isna().sum() #проверяем что в таблице больше нет пропусков

### 2.3 Дубликаты

In [ ]:
df.duplicated().sum() #считаем явные дубликаты в таблице

In [ ]:
df = df.drop_duplicates() #удаляем явные дубликаты

In [ ]:
df.duplicated().sum() #проверяем что явных дубликатов в таблице не осталось

Теперь избавимся от неявных дубликатов в колонке `genre`. Например, название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

In [ ]:
df['genre'].sort_values().unique()  #изучаем уникальные значения столбца

В процессе изучения списка были найдены неявные дубликаты названия `hiphop`. Это названия с ошибками или альтернативные названия того же жанра.

Были найдены следующие неявные дубликаты:
* *hip*,
* *hop*,
* *hip-hop*.

In [ ]:
df['genre'] = df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop')  #заменяем дубликаты

In [ ]:
df['genre'].sort_values().unique() #проверяем корректность изменений

### Промежуточный вывод

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Мы исправили заголовки, чтобы упростить работу с таблицей. Без дубликатов исследование станет более точным.

Пропущенные значения мы заменили на `'unknown'`, а также обработали явные и неявные дубликаты.

## 3. Проверка гипотез

### 3.1 Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.



In [ ]:
df.groupby('city')['city'].count() 
#группируем данные по городу и подсчитываем прослушивания в каждой группе

Прослушиваний в Москве - 45360, в Петербурге - 19719. В Москве прослушиваний значительно больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [ ]:
df.groupby('day')['day'].count() 
#группируем данные по дню недели и считаем прослушивания в понедельник, среду и пятницу

В среднем пользователи из двух городов менее активны по средам (понедельник - 22697 прослушиваний, среда - 19233 прослушивания, пятница - 23149 прослушиваний). Но картина может измениться, если рассмотреть каждый город в отдельности.

In [ ]:
def number_tracks(day, city): #создаём функцию для подсчёта прослушиваний для конкретного города и дня
    track_list = df[df['city'] == city]
    track_list = track_list[track_list['day'] == day]
    track_list_count = track_list['user_id'].count()
    return track_list_count

In [ ]:
for day in data['day'].unique():
    for city in data['city'].unique():
        print(f'Число прослушиваний для города {city} в день {day} равно {number_tracks(day, city)}')

**Получаем следующие результаты:**
* Понедельник, Москва - 16715 прослушиваний
* Среда, Москва - 11755 прослушиваний
* Пятница, Москва - 16890 прослушиваний
* Понедельник, Петербург - 56982 прослушивания
* Среда, Петербург - 7478 прослушиваний
* Пятница, Петербург - 6259 прослушиваний

In [ ]:
table = pd.DataFrame(data=[['Moscow', 16715, 11755, 16890], ['Saint-Petersburg', 5982, 7478, 6259]], columns=['city', 'monday', 'wednesday', 'friday'])
#создаём датафрейм со столбцами 'city', 'monday', 'wednesday', 'friday' и заполняем его получеными данными

### Промежуточный вывод

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### 3.2 Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [ ]:
moscow_general = df[df['city'] == 'Moscow'] #собираем данные по Москве в отдельный датафрейм

In [ ]:
spb_general = df[df['city'] == 'Saint-Petersburg'] #собираем данные по Петербургу в отдельный датафрейм

In [ ]:
#создаём функцию которая будет возвращать информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]

Cравним результаты функции genre_weekday() для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00)

In [ ]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

In [ ]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

In [ ]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

In [ ]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

### Промежуточный вывод

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.3 Жанровые предпочтения в Москве и Петербурге

In [ ]:
moscow_genres = moscow_general.groupby('genre')['genre'].count()
moscow_genres = moscow_genres.sort_values(ascending=False) 
#группируем таблицу `moscow_general` по жанру и считаем прослушивания треков каждого жанра

In [ ]:
moscow_genres.head(10) #проверяем первые 10 строк moscow_genres

In [ ]:
spb_genres = spb_general.groupby('genre')['genre'].count()
spb_genres = spb_genres.sort_values(ascending=False)
#группируем таблицу `spb_general` по жанру и считаем прослушивания треков каждого жанра

In [ ]:
spb_genres.head(10) #проверяем первые 10 строк spb_genres

### Промежуточный вывод
Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.

## Итоги исследования

Мы проверили три гипотезы и установили:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.